In [1]:
!nvidia-smi

Wed Dec 27 01:33:10 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install albumentations

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 26.3/166.8 GB disk)


In [6]:
from ultralytics import YOLO
from IPython.display import display, Image

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cwd = '/content/drive/MyDrive/SunflowerStarRebirth/StarChamber'

In [9]:
import os
os.chdir(cwd)

In [10]:
!dir models

StarSeg_Yolov8.pt


In [11]:
path_model = 'models/StarSeg_Yolov8.pt'
os.path.exists(path_model)

True

In [12]:
model = YOLO(path_model)
model.info()

In [14]:
path_data = 'data/cold_box_flips/flipping_data.csv'
os.path.exists(path_data)

True

In [15]:
import pandas as pd
data = pd.read_csv(path_data)
data

,video_path,time_order,cohort,id,run,week,temp_c,start_frame,end_frame,time_s,flip_order,type,eating_before,sleeping_before
0,data/cold_box_flips/R3_W1_12C/i12/0-12_14_23-1...,1,i,i12,R3,W1,12.0,27,30,1.5,NaN,Being flipped,n,m
1,data/cold_box_flips/R3_W1_12C/i12/0-12_14_23-1...,2,i,i12,R3,W1,12.0,31,131,50.0,0.0,Upside Down,n,m
2,data/cold_box_flips/R3_W1_12C/i12/0-12_14_23-1...,3,i,i12,R3,W1,12.0,132,149,8.5,NaN,Right Side Up,n,m
3,data/cold_box_flips/R3_W1_12C/i12/0-12_14_23-1...,4,i,i12,R3,W1,12.0,150,157,3.5,NaN,Being flipped,n,m
4,data/cold_box_flips/R3_W1_12C/i12/0-12_14_23-1...,5,i,i12,R3,W1,12.0,158,249,45.5,1.0,Upside Down,n,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,data/cold_box_flips/R1_W1_12C/p6/0-11_12_23-13...,35,p,p6,R1,W1,12.0,1030,1042,6.0,NaN,Being flipped,m,n
1798,data/cold_box_flips/R1_W1_12C/p6/0-11_12_23-13...,36,p,p6,R1,W1,12.0,1043,1077,17.0,12.0,Upside Down,m,n
1799,data/cold_box_flips/R1_W1_12C/p6/0-11_12_23-13...,37,p,p6,R1,W1,12.0,1078,1084,3.0,NaN,Right Side Up,m,n
1800,data/cold_box_flips/R1_W1_12C/p6/0-11_12_23-13...,38,p,p6,R1,W1,12.0,1085,1089,2.0,NaN,Being flipped,m,n


In [16]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm

from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict

In [ ]:
def add_flip_legend(im, text, color,
                    top_left_corner = (40,40),
                    rectangle = (100, 100),
                    rect_thickness = -1,
                    font = cv2.FONT_HERSHEY_SIMPLEX,
                    text_x_offset = 20,
                    text_y_offset = 20,
                    font_scale = 2.5,
                    line_type = 3):

  tl_rect = top_left_corner
  br_rect = (tl_rect[0] + rectangle[0], tl_rect[1] + rectangle[1])
  text_pos = (br_rect[0]+text_x_offset,
              int((tl_rect[1]+br_rect[1])/2 + text_y_offset))

  cv2.rectangle(im, tl_rect, br_rect, color, rect_thickness)
  cv2.putText(im, text, text_pos, font,
              font_scale, color, line_type)

def add_speed_up_text_to_img (im, speed_up,
                              font = cv2.FONT_HERSHEY_SIMPLEX,
                              font_scale = 3,
                              font_color = (0, 0, 255),
                              line_type = 3,
                              thickness = 1,
                              text_x_offset = 50,
                              text_y_offset = 50):
  text = str(speed_up) + 'x'
  text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
  text_x = text_x_offset
  text_y = im.shape[0] - text_y_offset

  cv2.putText(im, text, (text_x, text_y), font,
              font_scale, font_color, line_type)

color_key_type = {'Right Side Up':(7,193, 255),#(255, 193, 7),#'#FFC107', RGB, BGR, hex
                  'Being flipped':(229, 136, 30),#(30, 136, 229),#'#1E88E5',
                  'Upside Down':(96, 27, 216),#(216,27,96),#'#D81B60',
                  None:(64, 77, 0),}#(0, 77, 64),}#'#004D40'}

flip_video = True
overwrite = True

record_fps = 2
speed_up = 20
fps = int(record_fps * speed_up)

n_videos = int(2*len(sorted([s for s in set(data['id'].tolist())])))-1
m = 0
for week in ['W1', 'W2']:
  for id in sorted([s for s in set(data['id'].tolist())]):
    tmp = data[(data['week'] == week) & (data['id'] == id)]
    if tmp.shape[0] > 0:
      video_path = tmp['video_path'].tolist()[0]
      print(f'({m}/{n_videos}) - {video_path}')

      video_dir = os.path.dirname(video_path)
      path_df = os.path.join(video_dir,
                            'rawTracks_yolov8_' + os.path.basename(video_path).split('.')[0] + '.pickle')
      if os.path.exists(path_df) != True or overwrite == True:

        cap = cv2.VideoCapture(video_path)
        l = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if flip_video:
          idxs = np.arange(0,l).tolist()
          types = [None]*l
          start_frames, end_frames, types_window = tmp['start_frame'].tolist(), tmp['end_frame'].tolist(), tmp['type'].tolist()

          for sf, ef, tp in zip(start_frames, end_frames, types_window):
            for i in range(sf, ef):
              types[i] = tp

          types_col = [color_key_type[s] for s in types]

        out_path = os.path.join(video_dir, 'yolov8_' + os.path.basename(video_path))
        out = cv2.VideoWriter(out_path,
                          cv2.VideoWriter_fourcc(*'mp4v'),
                          fps, (int(cap.get(3)), int(cap.get(4))))

        masks_xy = []
        ids = []
        n = 0
        pbar = tqdm(total=l, position=0, leave=True)
        while True:
            ret, im0 = cap.read()
            if not ret:
                break

            results = model.track(im0, persist=True, verbose=False)
            if results[0].masks != None and results[0].boxes.id != None:
              masks = results[0].masks.xy
              masks_xy.append(masks)
              track_ids = results[0].boxes.id.int().cpu().tolist()
              ids.append(track_ids)

              annotator = Annotator(im0, line_width=2)

              for mask, track_id in zip(masks, track_ids):
                if len(mask) > 0:
                  annotator.seg_bbox(mask=mask,
                                    mask_color=colors(track_id, True),
                                    track_label=str(track_id))
            else:
              masks_xy.append(None)
              ids.append(None)

            if flip_video:
              add_flip_legend(im0, types[n], types_col[n])

            add_speed_up_text_to_img (im0, speed_up)

            out.write(im0)
            pbar.update(n=1)
            n += 1
        pbar.close()

        df = pd.DataFrame(zip(ids, masks_xy), columns = ['ids', 'masks_xy'])
        df.to_pickle(path_df)
        del df

        out.release()
        cap.release()
    m += 1

(0/45) - data/cold_box_flips/R3_W1_12C/i1/0-12_14_23-14_00_42-i1.mp4


100%|██████████| 1440/1440 [03:35<00:00,  6.68it/s]


(1/45) - data/cold_box_flips/R3_W1_12C/i10/0-12_14_23-14_26_30-i10.mp4


100%|██████████| 1440/1440 [03:40<00:00,  6.53it/s]


(2/45) - data/cold_box_flips/R1_W1_12C/i11/0-11_12_23-14_51_47-i11.mp4


...

(32/45) - data/cold_box_flips/R1_W2_14-5C/i7/0-11_19_23-13_22_44-i7.mp4


100%|██████████| 1440/1440 [03:23<00:00,  7.06it/s]


(33/45) - data/cold_box_flips/R1_W2_14-5C/i8/0-11_19_23-13_37_10-i8.mp4


 61%|██████▏   | 882/1440 [02:17<01:27,  6.40it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-80-bbbd1c324518>", line 118, in <cell line: 50>
    df.to_pickle(path_df)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 3064, in to_pickle
    to_pickle(
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/pickle.py", line 97, in to_pickle
    with get_handle(
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/common.py", line 734, in get_handle
    check_parent_directory(str(handle))
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/common.py", line 596, in check_parent_directory
    if not parent.is_dir():
  File "/usr/lib/python3.10/pathlib.py", line 1305, in is_dir
    return S_ISDIR(self.stat().st_mode)
  File "/usr/lib/python3.10/pathlib.py", line 1097, in stat
    return self._accessor.stat(self, follow_symlinks=